In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [40]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://naver.com')

driver.find_element_by_id('query').send_keys('동성제약')
driver.find_element_by_id('search_btn').click()

driver.find_element_by_css_selector('div.lnb_menu').find_element_by_link_text('뉴스').click()

driver.find_element_by_id('_search_option_btn').click()

driver.find_element_by_css_selector('ul.option_menu').find_element_by_link_text('영역').click()

driver.find_element_by_css_selector('.menu.selected').find_element_by_link_text('제목').click()

In [64]:
title = []
date = []
company = []
context = []
reporter = []
data = {'title' : [], 'date' : [], 'company' : [], 'context' [], 'reporter' : []}
for news in driver.find_elements_by_css_selector('div.thumb'):
    news.click()
    html = news.page_source
    soup = bs(html, 'html.parser')
    data['title'].append(soup.select('font.head-title'))

[<selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d668e208c5", element="0.013954942651315694-8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4bc0623e66884cc03a0c55d

In [ ]:
driver.find_elements_by_css_selector('div.thumb')[0].click()

In [100]:
data = {'title' : [], 'date' : [], 'context' : [], 'reporter' : []}
driver.switch_to.window(driver.window_handles[1])
html = driver.page_source
soup = bs(html, 'html.parser')
data['title'].append(soup.select('font.headline-title')[0].text.strip())
data['date'].append(soup.select('li.date')[0].text.strip())
data['context'].append(soup.select(''))

In [101]:
title

'국민생활건강 파트너 ‘동성제약’, DDP에서 열린 ‘안티에이징&헬스페어’ 통해 염모제, 미백크림, 알러지컷 등 선보여'

In [83]:
a = {'justin' : [1]}
a['justin'].append(2)
a

{'justin': [1, 2]}

In [82]:
a['justin'].append(3)
a

{'justin': [1, 2, 3, 3, 3, 3]}

# 발생한 오류
### 1. 인터넷 뉴스 회사마다 html형식이 달라 크롤링에 제약이 생겼다.

# 새로 배운 것
### 1. Multiple class인 element 찾기
 - css_selector(.class.class)